In [58]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from datetime import datetime, timedelta

# TEST

In [260]:
df = pd.read_csv("AUD_CAD_M15_2005_202206.csv")
df.columns=['time', 'open', 'high', 'low', 'close']
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df

,time,open,high,low,close
0,2005-01-02T18:45:00.000000000Z,0.93884,0.94353,0.93884,0.94353
1,2005-01-02T19:15:00.000000000Z,0.94317,0.94415,0.94291,0.94353
2,2005-01-02T19:30:00.000000000Z,0.94353,0.94424,0.94317,0.94424
3,2005-01-02T19:45:00.000000000Z,0.94362,0.94460,0.94362,0.94460
4,2005-01-02T20:00:00.000000000Z,0.94406,0.94433,0.94308,0.94308
...,...,...,...,...,...
440863,2022-06-30T22:30:00.000000000Z,0.88846,0.88858,0.88822,0.88853
440864,2022-06-30T22:45:00.000000000Z,0.88848,0.88872,0.88802,0.88805
440865,2022-06-30T23:00:00.000000000Z,0.88803,0.88864,0.88780,0.88852
440866,2022-06-30T23:15:00.000000000Z,0.88852,0.88856,0.88816,0.88828


In [264]:
# calculate sma
df['sma'] = df['close'].rolling(20).mean()

# calculate standard deviation
df['sd'] = df['close'].rolling(20).std()

# calculate lower band
df['lb'] = df['sma'] - 2 * df['sd']

# calculate upper band
df['ub'] = df['sma'] + 2 * df['sd']


In [153]:
# plotting close prices with bollinger bands
dfpl=df[:40]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['sma'],line_color='blue'))
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['lb'],line_color='green'))
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['ub'],line_color='red'))
# fig.add_trace = px.line(dfpl, x='time', y=['close', 'sma', 'lb', 'ub'])
fig.show()


# Mark the candles

In [276]:
def event(high,low, lower_band, upper_band, middle_band,l):
    # Outside the lower BB

    def event_1(high, lower_band):
        if high < lower_band:
            return 1
        else:
            return 0

    df['bb_event1'] = np.vectorize(event_1)(high, lower_band)

    # Outside the upper BB
    def event_2(low, upper_band):
        if low > upper_band:
            return 1
        else:
            return 0

    df['bb_event2'] = np.vectorize(event_2)(low, upper_band)

    # Touches the lower BB
    def event_3(high, lower_band):
        if 0.9999*lower_band < high < 1.0001*lower_band:
            return 1
        else:
            return 0

    df['bb_event3'] = np.vectorize(event_3)(high, lower_band)


    # Touches the upper BB
    def event_4(low, upper_band):
        if 0.9999*upper_band < low < 1.0001*upper_band:
            return 1
        else:
            return 0

    df['bb_event4'] = np.vectorize(event_4)(low, upper_band)

    # Touches the middle BB from Top
    def event_5(high, middle_band):
        if 0.9999*middle_band < high < 1.0001*middle_band:
            return 1
        else:
            return 0

    df['bb_event5'] = np.vectorize(event_5)(high, middle_band)

    # Touches the middle BB from Bottom
    def event_6(low, middle_band):
        if 0.9999*middle_band < low < 1.0001*middle_band:
            return 1
        else:
            return 0

    df['bb_event6'] = np.vectorize(event_6)(low, middle_band)

    # Crosses the middle BB from Top towards Bottom
    def event_7(high,low, middle_band,l): 
        if l-1<=19:
            return 0
        if low[l]<middle_band[l] and middle_band[l]<high[l] and middle_band[l+1]<middle_band[l] and middle_band[l-1]>middle_band[l]:
            return 1
        else:
            return 0
    df['bb_event7'] = df.apply(lambda l: event_7(high,low, middle_band,l.name), axis=1)

    # Crosses the middle BB from Bottom towards Top
    def event_8(high,low, middle_band,l): 
        if l-1<=19:
            return 0
        if low[l]<middle_band[l] and middle_band[l]<high[l] and middle_band[l+1]>middle_band[l] and middle_band[l-1]<middle_band[l]:
            return 1
        else:
            return 0
    df['bb_event8'] = df.apply(lambda l: event_8(high,low, middle_band,l.name), axis=1)
    
    

# Save features

In [277]:
import pandas as pd
import os
import glob
  
csv_files = glob.glob(os.path.join('bb/', "*.csv"))
  
for f in csv_files:
      
    # read the csv file
    df = pd.read_csv(f)
    # calculate sma
    df['sma'] = df['close'].rolling(20).mean()

    # calculate standard deviation
    df['sd'] = df['close'].rolling(20).std()

    # calculate lower band
    df['lb'] = df['sma'] - 2 * df['sd']

    # calculate upper band
    df['ub'] = df['sma'] + 2 * df['sd']

    # detect the signal and save
    event(df.high,df.low, df.lb, df.ub, df.sma,len(df.high))
    col_n = ['bb_event1','bb_event2','bb_event3','bb_event4','bb_event5','bb_event6','bb_event7','bb_event8'] 
    a = pd.DataFrame(df,columns = col_n)
    a.to_csv('bb_results/'+f[3:],index=False)

